# Potato Nematode Predictor
This work contains the public sector consultancy work on a potato nematode predictor carried out by Aarhus University.

Start by configuring the notebook:

In [1]:
import wget
import geopandas
import qgrid

from pathlib import Path
from zipfile import ZipFile

%load_ext autotime
%load_ext autoreload
%autoreload 2

proj_path = Path.cwd().parent

Download the field polygons from The Danish Agricultural Agency:

In [10]:
dest_folder = proj_path / 'data' / 'external'
file_url_mapping = {
    'FieldPolygons2016.zip': 'http://i3.ytimg.com/vi/J---aiyznGQ/mqdefault.jpg'
}

for filename, url in file_url_mapping.items():
    dest_file = proj_path / 'data' / 'external' / filename
    if not dest_file.exists():
        wget.download(url, str())
        print("File has been downloaded: " + filename)
    else:
        print("File already exists: " + str(proj_path / 'data' / 'external' / filename))

File already exists: /home/jovyan/work/PublicSectorConsultancy/potato_nematode_predictor/data/external/FieldPolygons2016.zip
time: 16.5 ms


Then extract the zipfiles:

In [3]:
for zipfile in (proj_path / 'data' / 'external').glob('**/*.zip'):
    dest_folder = proj_path / 'data' / 'raw' / zipfile.stem
    if not dest_folder.exists():
        with ZipFile(str(zipfile), 'r') as zipObj:
            zipObj.extractall(str(dest_folder))
        print("Zipfile has been extracted: " + str(zipfile))
    else:
        print("Zipfile has already been extracted: " + str(zipfile))

Zipfile has already been extracted: /home/jovyan/work/PublicSectorConsultancy/potato_nematode_predictor/data/external/FieldPolygons2016.zip
time: 13.5 ms


Now load the shapefiles into geopandas dataframes and look at 3 example fields:

In [11]:
df_fields_2016 = geopandas.read_file(str(proj_path / 'data' / 'raw' / 'FieldPolygons2016' / 'Marker_2016_CVR.shp'))
df_fields_2016 = df_fields_2016.dropna()
df_fields_2016.head(3)

,Marknr,IMK_areal,Journalnr,CVR,Afgkode,Afgroede,GB,geometry
0,1-1,1.92,16-0033095,86378019,1.0,Vårbyg,1.0,"POLYGON ((680104.9238350778 6182870.688777743,..."
1,3-0,2.29,16-0039573,52534828,22.0,Vinterraps,1.0,"POLYGON ((536176.5556502932 6083754.099449905,..."
2,6-0,14.80,16-0018594,13308276,22.0,Vinterraps,1.0,"POLYGON ((668055.3661564761 6128190.005961547,..."


time: 38 s


Find the potato fields and count the number of unique sorts:

In [12]:
df_potato_2016 = df_fields_2016[df_fields_2016['Afgroede'].str.contains("kartof", case=False)]  

for crop_type in df_potato_2016['Afgroede'].unique():
    num_fields = df_potato_2016[df_fields_2016['Afgroede'] == crop_type].shape[0]
    print("There are " + str(num_fields)  + " fields of crop type: " + crop_type)

There are 3699 fields of crop type: Kartofler, stivelses-
There are 2655 fields of crop type: Kartofler, spise-
There are 1025 fields of crop type: Kartofler, lægge-
There are 539 fields of crop type: Kartofler, andre
time: 799 ms
